<a href="https://colab.research.google.com/github/markNZed/GPT-NeoX-Colab/blob/main/notebooks/codecompletion_benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# We could modify these paths to "stub" behavior for test/dev
workspaceDir = "/content"
GPTNeoXColabDirName = "GPT-NeoX-Colab"
GPTNeoXColabDir = f"{workspaceDir}/{GPTNeoXColabDirName}"

# Clone CodeXGLUE Repo

In [2]:
!git clone --depth 1 https://github.com/microsoft/CodeXGLUE.git

Cloning into 'CodeXGLUE'...
remote: Enumerating objects: 3373, done.
remote: Counting objects: 100% (3372/3372), done.
remote: Compressing objects: 100% (1534/1534), done.
remote: Total 3373 (delta 1748), reused 3326 (delta 1733), pack-reused 1 (from 1)
Receiving objects: 100% (3373/3373), 213.15 MiB | 17.56 MiB/s, done.
Resolving deltas: 100% (1748/1748), done.
Updating files: 100% (400/400), done.


In [8]:
#@title Clone GPT-NeoX-Colab
%%time
%cd {workspaceDir}
# Don't use --depth 1 because that does not play nice with git-annex
!git clone https://github.com/markNZed/GPT-NeoX-Colab.git
%cd {GPTNeoXColabDir}
%pip install -q -r requirements_colab.txt
%pip install --use-feature=fast-deps -q .
from dotenv import load_dotenv
import os
load_dotenv(f"{GPTNeoXColabDir}/.env")
import GPTNeoXColab
GPTNeoXColab.utils.colab.fetch_data("data/codecompletion/token_completion.tar.gz")
#GPTNeoXColab.utils.colab.fetch_data("models/codecompletion.tar.gz")
GPTNeoXColab.utils.colab.fetch_data("models/codecompletion/global_step7000_HF.tar.gz")
#%cd {GPTNeoXColabDir}/data/codecompletion
#!tar -xzf token_completion.tar.gz
%cd {GPTNeoXColabDir}/models/codecompletion
!tar -xzf global_step7000_HF.tar.gz
!mv global_step7000_HF latest

/content
fatal: destination path 'GPT-NeoX-Colab' already exists and is not an empty directory.
/content/GPT-NeoX-Colab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Data retrieval successful.
Data retrieval successful.
/content/GPT-NeoX-Colab/models/codecompletion
CPU times: user 107 ms, sys: 19.4 ms, total: 126 ms
Wall time: 14.8 s


# Using Byte-Pair Encoding Tokenizer

In [4]:
%cd {workspaceDir}
!wget https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-vocab.json &
!wget https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-merges.txt

/content
--2024-11-14 20:32:51--  https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-vocab.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.138.64, 52.216.109.5, 16.182.70.80, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.138.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1042301 (1018K) [application/json]
Saving to: ‘gpt2-vocab.json’

gpt2-vocab.json     100%[===================>]   1018K  1.05MB/s    in 0.9s    

2024-11-14 20:32:53 (1.05 MB/s) - ‘gpt2-vocab.json’ saved [1042301/1042301]

--2024-11-14 20:32:53--  https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-merges.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.138.64, 52.216.109.5, 16.182.70.80, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.138.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 456318 (446K) [text/plain]
Saving to: ‘gpt2-merges.txt’

gpt2-merges.txt     100%[===================>] 445.62K   

# HuggingFace Inference

In [9]:
from transformers import GPTNeoXForCausalLM, GPT2Tokenizer
import torch

%cd {workspaceDir}

# Initialize the tokenizer with your vocabulary and merge files
tokenizer = GPT2Tokenizer(vocab_file=f"{workspaceDir}/gpt2-vocab.json", merges_file=f"{workspaceDir}/gpt2-merges.txt")

# Load your model
model_path = f"{GPTNeoXColabDir}/models/codecompletion/latest"
model = GPTNeoXForCausalLM.from_pretrained(model_path)

# Set the model to evaluation mode
model.eval()

# Prompt the user for input
input_text = """<s> import sys , os <EOL> import imp <EOL> from optparse import make_option <EOL> from django . conf import settings <EOL> from django"""

# Tokenize and prepare input
input_ids = torch.tensor([tokenizer.encode(input_text)], dtype=torch.long)
attention_mask = torch.ones_like(input_ids)  # Create an attention mask for non-padded input

# Generate text with specified pad_token_id and attention_mask
with torch.no_grad():
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=200,          # Adjust this for desired output length
        temperature=0.7,        # Controls creativity
        top_k=50,               # Controls diversity
        top_p=0.9,              # Nucleus sampling
        num_return_sequences=1, # Number of sequences to return
        pad_token_id=model.config.eos_token_id,  # Set pad_token_id explicitly
        do_sample=True           # Enable sampling mode to use temperature and top_p
    )

# Decode the generated text
generated_text = tokenizer.decode(output[0].tolist())

# Function to replace special tokens with original representations
def replace_special_tokens(text):
    replacements = {
        "<EOL>": "\n",  # Replace with actual newline
        "<s>": "",
        "</s>": "",     # Remove end token
        "<STR_LIT>": "STR_LITERAL",  # Example replacement, adjust as necessary
        "<NUM_LIT>": "NUM_LITERAL",   # Example replacement, adjust as necessary
    }

    for token, replacement in replacements.items():
        text = text.replace(token, replacement)

    return text.strip()  # Strip leading/trailing whitespace

# Replace special tokens in the generated text
final_text = replace_special_tokens(generated_text)

# Print the final output
print("Generated text:", final_text)


/content
Generated text: import sys , os 
 import imp 
 from optparse import make_option 
 from django . conf import settings 
 from django . conf . urls import url 
 from django . template import RequestContext , RequestContext 
 from django . template import RequestContext , RequestContext , RequestContext , RequestContext 
 from django . template import RequestContext , RequestContext , RequestContext , RequestContext 
 from django . template import RequestContext , RequestContext 
 from django . template import RequestContext , RequestContext , RequestContext 
 from django . template import RequestContext 
 from django . template import RequestContext 
 from django . template import RequestContext 
 from django . template import RequestContext 
 from django . template import RequestContext <


In [5]:
%pip show datasets
%pip install datasets==1.18.0
%pip install hf-transfer
%pip install lm-eval --upgrade

In [10]:
!cp {workspaceDir}/gpt2-vocab.json {GPTNeoXColabDir}/models/codecompletion/vocab.json
!cp {workspaceDir}/gpt2-merges.txt {GPTNeoXColabDir}/models/codecompletion/merges.txt
!cp {GPTNeoXColabDir}/data/codecompletion/token_completion.tar.gz {workspaceDir}/CodeXGLUE/Code-Code/CodeCompletion-token/dataset/py150
%cd {workspaceDir}/CodeXGLUE/Code-Code/CodeCompletion-token/dataset/py150
!tar -xzf token_completion.tar.gz

/content/CodeXGLUE/Code-Code/CodeCompletion-token/dataset/py150


In [11]:
# This is not the way we are plannin gto run the model but I leave it hear for now. It needs a GPU but HF does not.
%cd {workspaceDir}/CodeXGLUE/Code-Code/CodeCompletion-token/code
!ls {GPTNeoXColabDir}/models/codecompletion
!python -u run_lm.py \
        --data_dir=../dataset/py150/token_completion \
        --lit_file=../dataset/py150/literals.json \
        --langs=$LANG \
        --output_dir=../dataset/py150 \
        --pretrain_dir=/content/GPT-NeoX-Colab/models/codecompletion \
        --log_file=../completion_python_eval.log \
        --model_type=gpt2 \
        --block_size=2048 \
        --do_eval \
        --per_gpu_eval_batch_size=4 \
        --logging_steps=100 \
        --seed=42

/content/CodeXGLUE/Code-Code/CodeCompletion-token/code
global_step7000_HF.tar.gz  latest  merges.txt  vocab.json
2024-11-14 20:37:31.759786: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-14 20:37:31.774528: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-14 20:37:31.778750: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-14 20:37:32.879915: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
11/14/2024 20:37:34 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits train